this script creates a final patient list used for Volume rendering project

In [1]:
import os
import numpy as np
import function_list_VR as ff
import pandas as pd

In [2]:
main_data_path = '/Volumes/McVeighLab/wip/zhennong/'
main_info_path = '/Users/zhennongchen/Documents/Zhennong_CT_Data/Patient_Overview'
patient_list = ff.find_all_target_files(['Abnormal/*','Normal/*'],os.path.join(main_data_path,'nii-images'))
normal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/normal_candidates.csv'))
abnormal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/abnormal_candidates.csv'))
entire_info = [normal_info,abnormal_info]
timeframe_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/time_frame_assignment_for_pretrained_AI_prediction.csv'))

In [34]:
print(patient_list.shape)

(375,)


In [39]:
normal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/normal_candidates.csv'))
abnormal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/abnormal_candidates.csv'))
entire_info = [normal_info,abnormal_info]

In [48]:
for count in range(0,len(patient_list)):
    p = patient_list[count]
    patient_id = os.path.basename(p)
    patient_class = os.path.basename(os.path.dirname(p))
    
    if patient_class == 'Abnormal':
        info = entire_info[1]
    else:
        info = entire_info[0]
    
    assert (patient_id in info.Patient_ID.values) == True
    assert (patient_id in timeframe_info.Patient_ID.values) == True
    
    t = timeframe_info.loc[timeframe_info['Patient_ID'] == patient_id]
    t = t.iloc[:,1:t.shape[1]]
   # print(t)
    
    i = info.loc[info['Patient_ID'] == patient_id]
    i = i.iloc[:,1:i.shape[1]]
    #print(i)
   
    
    if count == 0:
        final_list = pd.merge(t, i, on='Patient_ID')
    else:
        final_list = pd.concat([final_list,pd.merge(t, i, on='Patient_ID')],axis = 0)
    #print(final_list)
    

In [49]:
print(final_list.iloc[0])

Patient_Class                                                    Abnormal
Patient_ID                                                  CVC1801161735
ED/ES                                                                  ED
Timeframe_picked                                                        0
Total_timeframes                                                       10
Keywords                ['device', 'akinesi', 'function', 'thinning', ...
Highlight text          CTA CARDIAC MORPHOLOGY 57 - year - old female ...
Report text             Please  note  the  following  correction  to  ...
Acession                                                         50272821
Manufacturer                                           GE MEDICAL SYSTEMS
Model                                                       Revolution CT
Sex                                                                     F
Age                                                                  057Y
Protocol                              

In [51]:
# add one empty column
final_list.insert(2,'ZC_notes','')
print(final_list.columns.values,final_list.shape)

['Patient_Class' 'Patient_ID' 'ZC_notes' 'ED/ES' 'Timeframe_picked'
 'Total_timeframes' 'Keywords' 'Highlight text' 'Report text' 'Acession'
 'Manufacturer' 'Model' 'Sex' 'Age' 'Protocol' 'Directories_Full'
 'Directories_Function' 'Timeframes'] (375, 18)


In [52]:
final_list.reset_index(drop=True,inplace=True)
final_list.to_excel(os.path.join(main_info_path,'Case_search_list/Final_patient_list.xlsx'),index=True)

## demographics

In [3]:
main_info_path = '/Users/zhennongchen/Documents/Zhennong_CT_Data/Patient_Overview'
final_list = pd.read_csv(os.path.join(main_info_path,'Case_search_list/Final_patient_list.csv'))

In [4]:
#gender
male = 0
for i in range(0,final_list.shape[0]):
    case = final_list.iloc[i]
    if case['Sex'] == 'M':
        male += 1
        
print(male)
print(male/final_list.shape[0])
    

239
0.6373333333333333


In [5]:
# age
from statistics import mean
age = []
for i in range(0,final_list.shape[0]):
    case = final_list.iloc[i]
    if len(case['Age']) == 4:
        age.append(int(case['Age'][1])*10 + int(case['Age'][2]))
    
age = np.asarray(age)
print(age.mean(),age.std())
           

62.11764705882353 15.50925405390561


## WMA info

In [16]:
main_info_path = '/Users/zhennongchen/Documents/Zhennong_CT_Data/Patient_Overview'
final_list = pd.read_csv(os.path.join(main_info_path,'Case_search_list/Final_patient_list.csv'))
final_list_exclude = pd.read_csv(os.path.join(main_info_path,'Case_search_list/Final_patient_list_exclude.csv'))
final_list = final_list.fillna('')
final_list_exclude = final_list_exclude.fillna('')
print(final_list.shape)

(387, 26)


In [18]:
# WMA info count
countR = 0
countG = 0
countEF = 0

for i in range(0,final_list.shape[0]):
    case = final_list.iloc[i]
        
    info = case['Regional/Global/EF2'] 
    
    if ('R' in info):
        countR += 1
    if ('G' in info):
        countG += 1
        
    if ('E' in info):
        countEF += 1
    
print(countR,countG,countEF)
        

102 61 131
